<a href="https://colab.research.google.com/github/LorenzoTarricone/Advnced-Programming-and-Optimization-Algorithm/blob/main/Tarricone3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3
Lorenzo Tarricone (3141667)

Use the PuLP library https://pypi.org/project/PuLP/ to solve the following problems. Documentation to PuLP can be found here: https://coin-or.github.io/pulp/main/index.html

###Bakery problem (20 points)
Consider a small bakery with a single oven. It needs to schedule baking of n pastries, each of them having three requirements:

time when the preparations are done and pastry is ready for baking
time needed for baking, i.e., for how long should it remain in the oven
deadline: time when the custommer comes to pick up the pastry
At each moment, only one kind of pastry can be present in the oven.
Use an ILP to find a shortest baking schedule. Schedule, in this context, is a set of starting times s1, ..., sn denoting when should each pastry be put into oven. Note: these times need not be integral. However, integral variables will be useful to enforce that the periods when two different kinds of pastries are in the oven do not overlap.

Let us denote $e_1$, ..., $e_n$ the ending times of baking of each of the pastries, i.e., $e_i = s_i + baking time of pastry_i$. We need to make sure that for each two pastries $i,j$, one of the following needs to be true: $e_i ≤ s_j$ or $e_j ≤ s_i$. Obviously, they cannot hold at the same time and it depends on the precedence between i and j which one is true. Since we do not know the precedence in advance, which of these constraints should we include in the LP?

###Big-M method
This name usually refers to an alternative way how to start the simplex method without knowledge of the initial basic feasible solution. We did not cover this in the class and I do not go into details of this here either. But the other meaning of Big-M is a method for switching someof the constraints on/off depending on the value of some binary variable.

Imagine, we have variable x which should be bounded by $10$ if and only if some binary variable $z$ is set to zero. Also, assume that there is no reason to increase x beyound some large number $M$ (e.g., because we are minimizing over $x$, or we know that no feasible solution can have $x > M$ for some other reasons). Then, we can write $x ≤ 10 + Mz$: if $z$ is $0$, this switches the constraint ON. If $z=1$, this constraint evaluates to $x ≤ 10 + M$ which, by choice of $M$ is satisfied by any reasonable solution to our LP and this effectively switches the costraint OFF. Usually, due to possible numerical issues, it is recommended to use $M$ as small as possible. You can check the following blog for more discussion of big-M: https://orinanobworld.blogspot.com/2011/07/perils-of-big-m.html.

You may check that there is a suitable choice of $M$ in our problem and use this approach in your solution.

###Input
Text file containing a single line for each kind of pastry consisting of four numbers (integers) separated by spaces:

`ID PRE DLN BAK`

`ID` denotes the numerical `ID` of the pastry, `PRE` the time since midnight since when the pastry is ready for baking, `BAK` is the time it needs to spend in the oven, and `DLN` is the deadline when the pastry needs to be surely finished.
All times are in seconds.

###Output
List of starting times of each pastry. Should look like this:
```
s1: 23.0
s2: 4.0
s3: 25.0
s4: 72.0
s5: 34.0
```
...
###Bakery problem visualization (10 points)
Use library matplotlib to visualize your solution suitably. I leave to your creativity how to do it, but it should be clear what are the moments when oven needs to be open, what pastry goes out and what should be put in. There are many other things to visualize: expected arrivals of custommers and times when each pastry is ready, critical preparations (which pastry needs special care to be prepared on time, otherwise it would delay the whole schedule, etc). The main criterion for evaluation of this will be clarity and information it provides.



---

## Solving the problem

We start by installing the packages needed for the manipulation of the data and for finding the solution to the problem

In [ ]:
!pip install pulp
from pulp import *
import numpy as np
import pandas as pd
# from itertools import product

     |████████████████████████████████| 14.2 MB 23.8 MB/s 


Running the chun of code below a botton called "Choose files" will appear, letting you choosing the file from your pc, please select 'bakery.txt' that is the file provided in the assignment

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving bakery.txt to bakery.txt


We create the table

In [ ]:
import io
data = pd.read_csv(io.BytesIO(uploaded['bakery.txt']), sep = " ", header = None )
data.columns = ["ID", "PRE", "DLN", "BAK"]

We visualize the table

In [ ]:
data

,ID,PRE,DLN,BAK
0,0,4800,9600,600
1,1,5400,27000,600
2,2,0,600,600
3,3,3000,5400,600
4,4,1200,4200,600
5,5,3000,10800,1200
6,6,9000,15000,1200
7,7,8400,14400,1200
8,8,1200,20400,1200
9,9,600,24600,1200


In [ ]:
#We estimate the big value M in such a way that is big enough, but not that big to cause problems 
M = max(data["DLN"]) + 1000 

Here we define all the other variables

In [ ]:
#We start by defining the problem and the relative variables
problem = LpProblem("Bakery", LpMinimize)

#Create the target variables
DUR = LpVariable("DUR", lowBound= 0, cat="Continuous")

#Create the starting time variables 
s = LpVariable.dicts("s", [i for i in range(17)], cat="Continuous")

#Create the auxiliary binary variables (also needed for the big M method)
z = LpVariable.dicts("z", [(i, j) for i in range(17) for j in range(17)], cat= "Binary") 


Here we define the constraints, solve and check the status of the problem

In [ ]:
#We define now the objective function
problem += (DUR, "Objective: we want to minimize the total duration of the process ")

#We add the constraints
for i in range(17):
  problem += s[i] >= data["PRE"][i]
  problem += s[i] <= (data["DLN"][i] - data["BAK"][i])


for j in range(17):
  for k in range(17):
    if j != k: 
      problem += s[j] >= (s[k] + data["BAK"][k] - M * z[j, k])
      problem += s[k] >= (s[j] + data["BAK"][j] - M + M * z[j, k])
  
for i in range(17):
  problem += DUR >= (s[i] + data["BAK"][i])

#We call the solver
problem.solve()
LpStatus[problem.status] 

'Optimal'

Here we print out the variables we are interested in

In [ ]:
print(f"The whole process will last until: {value(problem.objective)}")
for i in range(17):
  print(s[i], ":", s[i].varValue)

The whole process will last until: 22200.0
s_0 : 7800.0
s_1 : 19800.0
s_2 : 0.0
s_3 : 4800.0
s_4 : 1800.0
s_5 : 8400.0
s_6 : 11400.0
s_7 : 12600.0
s_8 : 17400.0
s_9 : 600.0
s_10 : 18600.0
s_11 : 15600.0
s_12 : 13800.0
s_13 : 20400.0
s_14 : 9600.0
s_15 : 5400.0
s_16 : 2400.0


##Plotting the results

Here i decided to use the interactive graph provided by the library 'ploty' that has an entire set of plotting functions dedicated to scheduling.

The idea is to let the user with his/her mouse go over any border of the small rectangles in the plot and this will provide the details for that exact starting or finish time. 
The color is a scale from green to red and the more the end of the baking is close to the deadline the 'redder' the little rectangle will be.

N.B. The method requires a coplete date to build the graph, I imagined that the process of baking starts at midnight of the $27^{th} March$, but this is a completely random date.

In [ ]:
#We proceed with some plotting 
!pip install chart_studio
import pandas as pd
import chart_studio.plotly as py
import plotly.figure_factory as ff
import datetime

#I define a dictionary where I'll save the data about times needed to build the graph
j_record={}
for i in range(17):
  var_value = s[i].varValue
  start_time=str(datetime.timedelta(seconds=var_value)) # convert seconds to hours, minutes and seconds
  end_time=str(datetime.timedelta(seconds=var_value + data['BAK'][i]))
  urgent = int((data['DLN'][i] - (var_value + data['BAK'][i]))/228)  
  j_record[i]=[start_time, end_time, urgent]

#The method requires a dataframe where the informations are stored, I use a for loop to iteratively add the data
df=[]
for j in range(17):
  df.append(dict(Task=f'Product {j}', Start='2022-03-27 %s'%(str(j_record[j][0])), Finish='2022-03-27 %s'%(str(j_record[j][1])), Urgent = int(j_record[j][2])))

#Actual plotting    
fig = ff.create_gantt(df, 
                      colors=['rgb(230,18,18)','rgb(43,191,27)'], 
                      index_col='Urgent',  
                      group_tasks=True, 
                      showgrid_x=True,
                      bar_width=0.5,
                      title='Bakery production (red products are the one which production end closer to the deadline)')
fig.show()
fig.write_html("interactive.html")